# Preprocessing data
## Creation of socio-demographic features

We first reshape the "ts-x-21.03.01" file (accessible at [here](https://www.bfs.admin.ch/bfs/fr/home/statistiques/catalogues-banques-donnees/tableaux.assetdetail.16484444.html)) to have one entry for each region ("CODE_REGION"). Data cover the 2020 period, but published in 2021

In [5]:
import pandas as pd
import os
from functools import reduce
import helpers
import glob
import numpy as np
from settings import folder_data

%load_ext autoreload
%autoreload 2

socio_demo = pd.read_csv(
    os.path.join(folder_data, "ts-x-21.03.01.csv"),
    sep=";",
    dtype={
        "PERIOD_REF": str,
        "PERIOD_COMP": str,
        "CODE_REGION": str,
        "REGION": str,
        "INDICATORS": str,
        "UNIT_MES": str,
        "VALUE": float,
        "STATUS": str,
    },
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Makes some tests
# Check that we have the same number of INDICATORS per region
assert socio_demo.groupby("CODE_REGION").apply(len).unique() == 41
assert socio_demo.groupby("REGION").apply(len).unique() == 41


In [7]:
# Reshape data
socio_demo_reshaped = pd.pivot_table(
    socio_demo,
    values=["VALUE", "UNIT_MES"],
    index="REGION",
    columns="INDICATORS",
    aggfunc=lambda x: x,
)
socio_demo_reshaped = socio_demo_reshaped.swaplevel(axis=1).sort_index(axis=1)

# Same unit for every measure
tmp = socio_demo_reshaped.xs("UNIT_MES", axis=1, level=1, drop_level=False)
assert len(tmp.apply(lambda x: x.unique(), axis=0)) == 1

# -> no need to keep track of the unit
# Reshape data, but keeping only the values
socio_demo_reshaped = pd.pivot_table(
    socio_demo,
    values="VALUE",
    index=["REGION", "CODE_REGION"],
    columns="INDICATORS",
    aggfunc=lambda x: x,
)
cols = [
    c
    for c in socio_demo_reshaped.columns
    if c.startswith("Ind_14") or c.startswith("Ind_11")
]
socio_demo_reshaped = socio_demo_reshaped.drop(columns=cols)


In [8]:
# Read meta data and rename the columns
indicators = pd.read_excel(
    os.path.join(folder_data, "ts-x-21.03.01-APPENDIX.ods"),
    engine="odf",
    sheet_name="INDICATORS",
    header=1,
    usecols=["INDICATORS", "FR"],
)
indicators_dict = indicators.set_index("INDICATORS").to_dict()["FR"]
socio_demo_reshaped = socio_demo_reshaped.rename(columns=indicators_dict).reset_index(
    level=0
)


Filling missing values for economic features with 2019 data (use files from the swiss atlas [here](https://www.atlas.bfs.admin.ch/maps/13/fr/16419_9077_9075_138/25613.html)). Since municipalities are merged over time, we need to track changes. The file "mapping_commune" is a dict-like file indicating the changes in the GDE code of newly merged municipalities (obtained using [this](https://www.agvchapp.bfs.admin.ch/fr/communes/query) website). 

In [9]:
tmp = []
files = [
    "25603_132",
    "25606_132",
    "25609_132",
    "25613_132",
    "25616_132",
    "25619_132",
]
# Read all files
for file in files:
    df = helpers.read_xlsx_from_atlas(
        os.path.join(folder_data, "economic", f"{file}.xlsx"), nrows=2213
    ).drop(columns="Regionsname")
    tmp.append(df)

reduce_func = lambda left, right: pd.merge(
    left, right, left_index=True, right_index=True, how="outer"
)

tmp = reduce(
    reduce_func,
    tmp,
)
# Fill with smallest number
economic = tmp.fillna(3)
economic


,Etablissements dans le secteur primaire,Etablissements dans le secteur secondaire,Etablissements dans le secteur tertiaire,Emplois dans le secteur primaire,Emplois dans le secteur secondaire,Emplois dans le secteur tertiaire
Regions-ID,,,,,,
CH,52706.0,96052.0,547613.0,159296.0,1092397.0,4071011.0
1,16.0,16.0,110.0,43.0,24.0,417.0
2,25.0,134.0,862.0,54.0,1335.0,5605.0
3,14.0,29.0,233.0,37.0,121.0,870.0
4,36.0,41.0,198.0,100.0,134.0,800.0
...,...,...,...,...,...,...
6806,11.0,9.0,16.0,30.0,187.0,31.0
6807,34.0,20.0,47.0,133.0,183.0,142.0
6808,69.0,28.0,66.0,199.0,88.0,382.0


In [10]:
# Aggregate data using the new municipalities
economic=helpers.fusion_commune(economic)
economic


,Etablissements dans le secteur primaire,Etablissements dans le secteur secondaire,Etablissements dans le secteur tertiaire,Emplois dans le secteur primaire,Emplois dans le secteur secondaire,Emplois dans le secteur tertiaire
Regions-ID,,,,,,
CH,52706.0,96052.0,547613.0,159296.0,1092397.0,4071011.0
1,16.0,16.0,110.0,43.0,24.0,417.0
2,25.0,134.0,862.0,54.0,1335.0,5605.0
3,14.0,29.0,233.0,37.0,121.0,870.0
4,36.0,41.0,198.0,100.0,134.0,800.0
...,...,...,...,...,...,...
3715,35.0,9.0,33.0,105.0,9.0,145.0
5399,24.0,15.0,49.0,50.0,38.0,105.0
6037,107.0,188.0,946.0,289.0,1073.0,5205.0


In [11]:
cols = economic.columns.str.startswith("Etablissements")
economic["Etablissements, total"] = economic.loc[:, cols].sum(axis=1)
cols = economic.columns.str.startswith("Emplois")
economic["Emplois, total"] = economic.loc[:, cols].sum(axis=1)
economic.columns = [f"Economie - {c}" for c in economic.columns]
economic


,Economie - Etablissements dans le secteur primaire,Economie - Etablissements dans le secteur secondaire,Economie - Etablissements dans le secteur tertiaire,Economie - Emplois dans le secteur primaire,Economie - Emplois dans le secteur secondaire,Economie - Emplois dans le secteur tertiaire,"Economie - Etablissements, total","Economie - Emplois, total"
Regions-ID,,,,,,,,
CH,52706.0,96052.0,547613.0,159296.0,1092397.0,4071011.0,696371.0,5322704.0
1,16.0,16.0,110.0,43.0,24.0,417.0,142.0,484.0
2,25.0,134.0,862.0,54.0,1335.0,5605.0,1021.0,6994.0
3,14.0,29.0,233.0,37.0,121.0,870.0,276.0,1028.0
4,36.0,41.0,198.0,100.0,134.0,800.0,275.0,1034.0
...,...,...,...,...,...,...,...,...
3715,35.0,9.0,33.0,105.0,9.0,145.0,77.0,259.0
5399,24.0,15.0,49.0,50.0,38.0,105.0,88.0,193.0
6037,107.0,188.0,946.0,289.0,1073.0,5205.0,1241.0,6567.0


In [12]:
cols = [
    "Economie - Emplois, total",
    "Economie - Emplois, secteur primaire",
    "Economie - Emplois, secteur secondaire",
    "Economie - Emplois, secteur tertiaire",
    "Economie - Etablissements, total",
    "Economie - Etablissements, secteur primaire",
    "Economie - Etablissements, secteur secondaire",
    "Economie - Etablissements, secteur tertiaire",
]

socio_demo_reshaped = socio_demo_reshaped.drop(columns=cols)
socio_demo_reshaped = pd.merge(
    socio_demo_reshaped, economic, how="inner", left_index=True, right_index=True
)
socio_demo_reshaped


,REGION,Population - Habitants,Population - Variation en %,Population - Densité de la population,Population - Part du groupe d'âge 0-19 ans,Population - Part du groupe d'âge 20-64 ans,Population - Part du groupe d'âge 65+ ans,Population - Etrangers,Population - Taux brut de nuptialité,Population - Taux brut de divortialité,...,"Construction, logement - Taux de logements vacants","Construction, logement - Nouveaux logements construits",Economie - Etablissements dans le secteur primaire,Economie - Etablissements dans le secteur secondaire,Economie - Etablissements dans le secteur tertiaire,Economie - Emplois dans le secteur primaire,Economie - Emplois dans le secteur secondaire,Economie - Emplois dans le secteur tertiaire,"Economie - Etablissements, total","Economie - Emplois, total"
4551,Aadorf,9047.0,11.912420,453.711133,20.404554,60.362551,19.232895,15.839505,5.750940,2.433090,...,3.107477,4.952167,47.0,96.0,436.0,110.0,1416.0,2015.0,579.0,3541.0
4001,Aarau,21773.0,11.673591,1764.424635,17.383916,64.414642,18.201442,20.938777,5.868897,2.310589,...,1.014446,21.957871,11.0,183.0,2493.0,31.0,3213.0,31731.0,2687.0,34975.0
301,Aarberg,4632.0,13.307241,584.110971,19.624352,59.045769,21.329879,12.888601,4.751620,1.943844,...,0.645161,3.492687,18.0,56.0,274.0,108.0,795.0,1971.0,348.0,2874.0
4271,Aarburg,8376.0,19.657143,1899.319728,20.475167,65.222063,14.302770,44.138013,4.706450,3.499668,...,4.814636,2.602876,6.0,69.0,348.0,12.0,1111.0,2586.0,423.0,3709.0
321,Aarwangen,4620.0,10.526316,466.666667,20.541126,60.129870,19.329004,15.909091,5.010347,1.742730,...,7.601810,20.159504,23.0,50.0,185.0,76.0,601.0,873.0,258.0,1550.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,Zwieselberg,329.0,21.851852,133.739837,27.051672,56.534954,16.413374,4.863222,3.025719,0.000000,...,0.000000,12.345679,7.0,3.0,20.0,22.0,28.0,37.0,30.0,87.0
2793,Zwingen,2453.0,12.677997,532.104122,20.423971,64.329393,15.246637,24.419079,6.595218,2.061006,...,2.053037,13.253527,4.0,55.0,135.0,9.0,582.0,522.0,194.0,1113.0
6011,Zwischbergen,74.0,-7.500000,0.860165,2.702703,81.081081,16.216216,27.027027,0.000000,13.245033,...,1.075269,0.000000,3.0,3.0,10.0,3.0,12.0,29.0,16.0,44.0
628,Zäziwil,1579.0,-0.816583,291.866913,21.342622,58.708043,19.949335,6.523116,3.150599,2.520479,...,2.051282,0.000000,17.0,31.0,57.0,86.0,197.0,190.0,105.0,473.0


## Creation of financial features
We use the data from 2018 (accessible [here](https://www.atlas.bfs.admin.ch/maps/13/fr/16601_9164_9202_7267/25887.html#) and [here](https://www.atlas.bfs.admin.ch/maps/13/fr/16602_9164_9202_7267/25889.html#) for taxable income) and 2020 (accessible [here](https://www.atlas.bfs.admin.ch/core/projects/13/xshared/xlsx/25654_132.xlsx) for social subsidies).

#### Subsidies

In [13]:
nrows = 2199
social_subsidy = pd.read_excel(
    os.path.join(folder_data, "economic", "25654_132.xlsx"),
    header=[2, 3],
    skiprows=[4],
    nrows=nrows,
    na_values="X",
)
social_subsidy.columns = [
    social_subsidy.columns[i][1] if i in [0, 1] else social_subsidy.columns[i][0]
    for i in range(4)
]
# Set to zero to change the type to int and then str
social_subsidy.loc[social_subsidy.Regionsname == "Suisse", "Regions-ID"] = 0
social_subsidy = social_subsidy.astype({"Regions-ID": int}).astype({"Regions-ID": str})
# Set to CH
social_subsidy.loc[social_subsidy.Regionsname == "Suisse", "Regions-ID"] = "CH"
social_subsidy = social_subsidy.set_index("Regions-ID")
social_subsidy


,Regionsname,Nombre de bénéficiaires,"Taux d'aide sociale, en %"
Regions-ID,,,
CH,Suisse,272052.0,3.161178
1,Aeugst am Albis,26.0,1.312470
2,Affoltern am Albis,392.0,3.186210
3,Bonstetten,63.0,1.130650
4,Hausen am Albis,47.0,1.253000
...,...,...,...
6806,Vendlincourt,14.0,2.504470
6807,Basse-Allaine,38.0,3.069470
6808,Clos du Doubs,34.0,2.692000


In [14]:
nrows = 2203
# Load the population of 2019 (here, https://www.bfs.admin.ch/bfs/fr/home/statistiques/catalogues-banques-donnees/tableaux.assetdetail.11587766.html)
pop_in2019 = pd.read_excel(
    os.path.join(folder_data, "pop_in_2019.xlsx"),
    header=[5],
    skiprows=[6, 7],
    nrows=nrows,
    na_values="*",
)
pop_in2019 = pop_in2019.rename(columns={"Code commune": "Regions-ID"})
pop_in2019.loc[pop_in2019.Commune == "Suisse", "Regions-ID"] = 0
pop_in2019 = pop_in2019.astype({"Regions-ID": int}).astype({"Regions-ID": str})
# Set to ch
pop_in2019.loc[pop_in2019.Commune == "Suisse", "Regions-ID"] = "CH"
pop_in2019 = pop_in2019[["Regions-ID", "Habitants"]].set_index("Regions-ID")
pop_in2019


,Habitants
Regions-ID,
CH,8544527
1,1982
2,12229
3,5548
4,3701
...,...
6806,540
6807,1230
6808,1286


In [15]:
# Fill missing values (less than 6 people according to the excel file)
tmp = social_subsidy["Nombre de bénéficiaires"]
social_subsidy["Nombre de bénéficiaires"] = tmp.where(
    ~social_subsidy["Nombre de bénéficiaires"].isnull(), 5
)

# Use population from 2020 if possible, otherwise use pop from 2019
tmp = social_subsidy["Taux d'aide sociale, en %"]
null_social_subsidy = social_subsidy["Taux d'aide sociale, en %"].isnull()
social_subsidy["Taux d'aide sociale, en %"] = tmp.where(
    ~null_social_subsidy,
    social_subsidy["Nombre de bénéficiaires"]
    / socio_demo_reshaped["Population - Habitants"]
    * 100,
)

tmp = social_subsidy["Taux d'aide sociale, en %"]
social_subsidy["Taux d'aide sociale, en %"] = tmp.where(
    ~null_social_subsidy,
    social_subsidy["Nombre de bénéficiaires"] / pop_in2019["Habitants"] * 100,
)
social_subsidy


,Regionsname,Nombre de bénéficiaires,"Taux d'aide sociale, en %"
Regions-ID,,,
CH,Suisse,272052.0,3.161178
1,Aeugst am Albis,26.0,1.312470
2,Affoltern am Albis,392.0,3.186210
3,Bonstetten,63.0,1.130650
4,Hausen am Albis,47.0,1.253000
...,...,...,...
6806,Vendlincourt,14.0,2.504470
6807,Basse-Allaine,38.0,3.069470
6808,Clos du Doubs,34.0,2.692000


In [16]:
social_subsidy["pop"] = (
    social_subsidy["Nombre de bénéficiaires"]
    / social_subsidy["Taux d'aide sociale, en %"]
    * 100
)
social_subsidy = social_subsidy.drop(columns="Taux d'aide sociale, en %")
social_subsidy


,Regionsname,Nombre de bénéficiaires,pop
Regions-ID,,,
CH,Suisse,272052.0,8.606033e+06
1,Aeugst am Albis,26.0,1.980998e+03
2,Affoltern am Albis,392.0,1.230302e+04
3,Bonstetten,63.0,5.572016e+03
4,Hausen am Albis,47.0,3.750998e+03
...,...,...,...
6806,Vendlincourt,14.0,5.590005e+02
6807,Basse-Allaine,38.0,1.237999e+03
6808,Clos du Doubs,34.0,1.263001e+03


In [17]:
# Aggregate data using the new municipalities
cols_to_agg = ["Nombre de bénéficiaires", "pop"]
social_subsidy=helpers.fusion_commune(social_subsidy,cols_to_agg)

social_subsidy["Taux d'aide sociale"] = (
    social_subsidy["Nombre de bénéficiaires"] / social_subsidy["pop"]
)
social_subsidy = social_subsidy.drop(columns=["pop"])
social_subsidy


,Regionsname,Nombre de bénéficiaires,Taux d'aide sociale
Regions-ID,,,
CH,Suisse,272052.0,0.031612
1,Aeugst am Albis,26.0,0.013125
2,Affoltern am Albis,392.0,0.031862
3,Bonstetten,63.0,0.011307
4,Hausen am Albis,47.0,0.012530
...,...,...,...
2238,NaN,31.0,0.014129
2430,NaN,38.0,0.032730
3715,NaN,20.0,0.056338


#### Taxable income/per taxpayer
Follow the same step as for the subsidies

In [18]:
mean_income_taxpayer = helpers.read_xlsx_from_atlas(
    os.path.join(folder_data, "economic", "25887_132.xlsx")
)


We use the [file](https://www.estv.admin.ch/estv/fr/accueil/afc/statistiques-fiscales/statistiques-fiscales-general/statistiques-impot-federal-direct/ifd-pp-communes-depuis-1983.html) to fill missing values. In particular, some categories ("C0","C1","C2","C3","C4","C5","C6") can have missing values. In that case, we set them to zero. Hence, our proportion is only a lower bound, but close to the correct proportion.

In [19]:
taxpayer = pd.read_excel(
    os.path.join(folder_data, "economic", "00_gemeindeauswertung_sp_2018.xlsx"),
    header=[12],
    sheet_name="241",
    na_values=["*", "* "],
    usecols=[1, 5, 6, 7, 8, 9, 10, 11],
)

taxpayer.columns = ["Regions-ID", "C0", "C1", "C2", "C3", "C4", "C5", "C6"]
tax_per_cat = taxpayer[[f"C{i}" for i in range(7)]].to_numpy().astype(float)
total = np.nansum(tax_per_cat, axis=1)
taxpayer["total"] = total
taxpayer.iloc[0, 0] = 0
taxpayer = taxpayer.astype({"Regions-ID": int})
taxpayer.iloc[0, 0] = "CH"
taxpayer = taxpayer.astype({"Regions-ID": str}).set_index("Regions-ID")
taxpayer = taxpayer[["total"]]
taxpayer


,total
Regions-ID,
CH,3791548.0
1,929.0
2,5441.0
3,2512.0
4,1639.0
...,...
6806,248.0
6807,493.0
6808,559.0


In [20]:
# Compute the number of taxpayer if data is available
mean_income_taxpayer["nb_contribuable"] = (
    mean_income_taxpayer["Revenu imposable, en mio. de francs"]
    * 1e6
    / mean_income_taxpayer["Revenu imposable par contribuable, en francs"]
)
mean_income_taxpayer = mean_income_taxpayer.drop(
    columns=["Revenu imposable par contribuable, en francs"]
)


In [21]:
# fill with the other method when missing
mean_income_taxpayer[
    "nb_contribuable"
] = mean_income_taxpayer.nb_contribuable.where(
    ~mean_income_taxpayer.nb_contribuable.isnull(), taxpayer.total
)
mean_income_taxpayer


,Regionsname,"Revenu imposable, en mio. de francs",nb_contribuable
Regions-ID,,,
CH,Suisse,298823.4537,3378409.0
1,Aeugst am Albis,101.1675,929.0
2,Affoltern am Albis,394.6064,5441.0
3,Bonstetten,226.1247,2512.0
4,Hausen am Albis,150.0025,1639.0
...,...,...,...
6806,Vendlincourt,12.9812,248.0
6807,Basse-Allaine,27.8383,493.0
6808,Clos du Doubs,29.0584,559.0


In [22]:
# Map the data to the new municipalities
cols_to_agg = ["Revenu imposable, en mio. de francs", "nb_contribuable"]

mean_income_taxpayer=helpers.fusion_commune(mean_income_taxpayer,cols_to_agg)
    
mean_income_taxpayer["Revenu imposable par contribuable, en francs"] = (
    mean_income_taxpayer["Revenu imposable, en mio. de francs"]
    / mean_income_taxpayer["nb_contribuable"]
    * 1e6
)
mean_income_taxpayer = mean_income_taxpayer.drop(columns=["nb_contribuable"])
mean_income_taxpayer


,Regionsname,"Revenu imposable, en mio. de francs","Revenu imposable par contribuable, en francs"
Regions-ID,,,
CH,Suisse,298823.4537,88450.940576
1,Aeugst am Albis,101.1675,108899.354144
2,Affoltern am Albis,394.6064,72524.609447
3,Bonstetten,226.1247,90017.794586
4,Hausen am Albis,150.0025,91520.744356
...,...,...,...
3715,NaN,7.5918,53842.553191
5399,NaN,16.8801,50238.392857
6037,NaN,490.4707,72954.142496


#### Taxable income/capita

In [23]:
mean_income_hab = helpers.read_xlsx_from_atlas(
    os.path.join(folder_data, "economic", "25889_132.xlsx")
)


In [24]:
# No missing values here, so we can compute the total population in 2018 and
# use it to take the proportions after aggregating the data
mean_income_hab["pop"] = (
    mean_income_hab["Revenu imposable, en mio. de francs"]
    * 1e6
    / mean_income_hab["Revenu imposable par habitant/-e, en francs"]
)
mean_income_hab = mean_income_hab.drop(
    columns=["Revenu imposable par habitant/-e, en francs"]
)


In [25]:
# Aggregate the data and compute the new ratio
cols_to_agg=["Revenu imposable, en mio. de francs", "pop"]
mean_income_hab=helpers.fusion_commune(mean_income_hab,cols_to_agg)
    
    
mean_income_hab["Revenu imposable par habitant/-e, en francs"] = (
    mean_income_hab["Revenu imposable, en mio. de francs"]
    / mean_income_hab["pop"]
    * 1e6
)
mean_income_hab = mean_income_hab.drop(columns=["pop"])
mean_income_hab


,Regionsname,"Revenu imposable, en mio. de francs","Revenu imposable par habitant/-e, en francs"
Regions-ID,,,
CH,Suisse,298823.4537,34972.498033
1,Aeugst am Albis,101.1675,51043.138244
2,Affoltern am Albis,394.6064,32268.084062
3,Bonstetten,226.1247,40757.876712
4,Hausen am Albis,150.0025,40530.262091
...,...,...,...
3715,NaN,7.5918,21385.352113
5399,NaN,16.8801,25269.610778
6037,NaN,490.4707,48494.235713


## Merging
Merging socio-demographic features and financial features together

In [26]:
tmp = pd.merge(
    socio_demo_reshaped,
    social_subsidy.drop(columns="Regionsname"),
    how="outer",
    left_index=True,
    right_index=True,
)
tmp = pd.merge(
    tmp,
    mean_income_taxpayer.drop(columns="Regionsname"),
    how="outer",
    left_index=True,
    right_index=True,
)
tmp = pd.merge(
    tmp,
    mean_income_hab.drop(columns="Regionsname"),
    how="outer",
    left_index=True,
    right_index=True,
)
tmp


,REGION,Population - Habitants,Population - Variation en %,Population - Densité de la population,Population - Part du groupe d'âge 0-19 ans,Population - Part du groupe d'âge 20-64 ans,Population - Part du groupe d'âge 65+ ans,Population - Etrangers,Population - Taux brut de nuptialité,Population - Taux brut de divortialité,...,Economie - Emplois dans le secteur secondaire,Economie - Emplois dans le secteur tertiaire,"Economie - Etablissements, total","Economie - Emplois, total",Nombre de bénéficiaires,Taux d'aide sociale,"Revenu imposable, en mio. de francs_x","Revenu imposable par contribuable, en francs","Revenu imposable, en mio. de francs_y","Revenu imposable par habitant/-e, en francs"
1,Aeugst am Albis,1981.0,8.607456,250.442478,19.787986,62.190813,18.021201,14.184755,7.065355,2.018673,...,24.0,417.0,142.0,484.0,26.0,0.013125,101.1675,108899.354144,101.1675,51043.138244
10,Obfelden,5721.0,22.087068,758.753316,23.579794,59.639923,16.780283,21.464779,4.946996,1.943463,...,287.0,996.0,320.0,1334.0,156.0,0.027268,184.6464,78740.469083,184.6464,32978.460439
100,Stadel,2293.0,15.983814,177.889837,21.020497,61.971217,17.008286,14.086350,6.547359,1.745962,...,69.0,372.0,152.0,528.0,40.0,0.017444,82.3573,76327.432808,82.3573,35979.598078
1001,Doppleschwand,789.0,9.888579,113.525180,26.742712,58.681876,14.575412,4.689480,5.092298,1.273074,...,56.0,88.0,73.0,229.0,5.0,0.006394,16.6286,56559.863946,16.6286,21264.194373
1002,Entlebuch,3280.0,-0.545785,57.644991,21.798780,59.664634,18.536585,7.012195,4.875076,1.218769,...,461.0,947.0,381.0,1781.0,39.0,0.011890,79.3894,57653.885258,79.3894,24174.604141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,Walliswil bei Wangen,606.0,3.412969,197.394137,15.676568,63.036304,21.287129,4.290429,3.294893,0.000000,...,11.0,75.0,31.0,107.0,7.0,0.011551,18.0062,63626.148410,18.0062,29615.460526
992,Wangen an der Aare,2321.0,13.330078,444.636015,19.000431,61.826799,19.172770,15.381301,5.180229,3.885172,...,249.0,823.0,207.0,1112.0,60.0,0.025851,70.0451,63160.595131,70.0451,30296.323529
993,Wangenried,405.0,2.015113,139.175258,20.000000,56.296296,23.703704,4.444444,2.409639,0.000000,...,46.0,37.0,34.0,118.0,5.0,0.011765,12.3146,61266.666667,12.3146,28975.529412
995,Wiedlisbach,2378.0,9.032554,317.066667,18.250631,63.456686,18.292683,17.872161,4.206099,1.261830,...,347.0,813.0,185.0,1208.0,77.0,0.032380,68.9439,63077.676121,68.9439,29004.585612


Merging also with the critical infrastructures

In [27]:
critical_infra=helpers.get_infra()
critical_infra

,train_station,bank,hospital,prod_plant,prod_plant_50MW,official_gov,airport_filtered,consumption,supermarket,airport,prod_plant_10MW
Regions-ID,,,,,,,,,,,
4206,2.0,2.0,0.0,3053.98,3053.98,0.0,0.0,0.000000,2.0,0.0,3053.98
3443,3.0,5.0,0.0,12724.67,12724.67,0.0,0.0,20625.081067,7.0,0.0,12724.67
6806,2.0,0.0,0.0,441.46,441.46,0.0,0.0,0.000000,0.0,0.0,441.46
6775,2.0,0.0,0.0,1381.70,1381.70,0.0,0.0,0.000000,1.0,0.0,1381.70
2554,2.0,0.0,0.0,1094.30,1094.30,0.0,0.0,0.000000,1.0,0.0,1094.30
...,...,...,...,...,...,...,...,...,...,...,...
2261,0.0,0.0,0.0,6.50,6.50,0.0,0.0,0.000000,0.0,0.0,6.50
708,0.0,0.0,0.0,26.00,26.00,0.0,0.0,0.000000,0.0,0.0,26.00
3713,0.0,0.0,0.0,189000.00,0.00,0.0,0.0,3525.475713,0.0,0.0,0.00


In [28]:
# Merge
final = pd.merge(tmp, critical_infra, how="outer", left_index=True, right_index=True)
# Fill empty cells with 0
final[critical_infra.columns] = final[critical_infra.columns].fillna(0)
# Sum over all rows except Schweiz
final.loc[final.REGION == "Schweiz", critical_infra.columns] = (
    final.loc[final.REGION != "Schweiz", critical_infra.columns].sum(axis=0).tolist()
)
final


,REGION,Population - Habitants,Population - Variation en %,Population - Densité de la population,Population - Part du groupe d'âge 0-19 ans,Population - Part du groupe d'âge 20-64 ans,Population - Part du groupe d'âge 65+ ans,Population - Etrangers,Population - Taux brut de nuptialité,Population - Taux brut de divortialité,...,bank,hospital,prod_plant,prod_plant_50MW,official_gov,airport_filtered,consumption,supermarket,airport,prod_plant_10MW
1,Aeugst am Albis,1981.0,8.607456,250.442478,19.787986,62.190813,18.021201,14.184755,7.065355,2.018673,...,0.0,0.0,1250.61,1250.61,0.0,0.0,0.000000e+00,1.0,0.0,1250.61
10,Obfelden,5721.0,22.087068,758.753316,23.579794,59.639923,16.780283,21.464779,4.946996,1.943463,...,1.0,0.0,1557.58,1557.58,2.0,0.0,9.793433e+03,3.0,0.0,1557.58
100,Stadel,2293.0,15.983814,177.889837,21.020497,61.971217,17.008286,14.086350,6.547359,1.745962,...,0.0,0.0,493.54,493.54,0.0,0.0,0.000000e+00,1.0,0.0,493.54
1001,Doppleschwand,789.0,9.888579,113.525180,26.742712,58.681876,14.575412,4.689480,5.092298,1.273074,...,0.0,0.0,595.00,595.00,0.0,0.0,0.000000e+00,0.0,0.0,595.00
1002,Entlebuch,3280.0,-0.545785,57.644991,21.798780,59.664634,18.536585,7.012195,4.875076,1.218769,...,1.0,0.0,8193.36,8193.36,0.0,0.0,0.000000e+00,2.0,0.0,8193.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,Walliswil bei Wangen,606.0,3.412969,197.394137,15.676568,63.036304,21.287129,4.290429,3.294893,0.000000,...,0.0,0.0,401.71,401.71,0.0,0.0,0.000000e+00,0.0,0.0,401.71
992,Wangen an der Aare,2321.0,13.330078,444.636015,19.000431,61.826799,19.172770,15.381301,5.180229,3.885172,...,1.0,0.0,2427.11,2427.11,0.0,0.0,1.081781e+04,3.0,0.0,2427.11
993,Wangenried,405.0,2.015113,139.175258,20.000000,56.296296,23.703704,4.444444,2.409639,0.000000,...,0.0,0.0,292.78,292.78,0.0,0.0,0.000000e+00,0.0,0.0,292.78
995,Wiedlisbach,2378.0,9.032554,317.066667,18.250631,63.456686,18.292683,17.872161,4.206099,1.261830,...,1.0,0.0,1469.56,1469.56,0.0,0.0,0.000000e+00,1.0,0.0,1469.56


In [29]:
assert np.allclose(
    final["Revenu imposable, en mio. de francs_y"],
    final["Revenu imposable, en mio. de francs_x"],
)
final = final.drop(columns=["Revenu imposable, en mio. de francs_y"])
final = final.rename(
    columns={
        "Revenu imposable, en mio. de francs_x": "Revenu imposable, en mio. de francs"
    }
)


In [30]:
mapping = pd.read_excel(
    os.path.join(
        folder_data, "Schweizerische Gemeinden und zuständige Stromnetzbetreiber.xlsx"
    ),
    header=2,
)
mapping = mapping.astype({"Gde-Nr.": str}).set_index(["Gde-Nr."])
final = pd.merge(
    final,
    mapping.groupby("Gde-Nr.").apply(len).to_frame("nb_gdr"),
    left_index=True,
    right_index=True,
    how="left",
)
final


,REGION,Population - Habitants,Population - Variation en %,Population - Densité de la population,Population - Part du groupe d'âge 0-19 ans,Population - Part du groupe d'âge 20-64 ans,Population - Part du groupe d'âge 65+ ans,Population - Etrangers,Population - Taux brut de nuptialité,Population - Taux brut de divortialité,...,hospital,prod_plant,prod_plant_50MW,official_gov,airport_filtered,consumption,supermarket,airport,prod_plant_10MW,nb_gdr
1,Aeugst am Albis,1981.0,8.607456,250.442478,19.787986,62.190813,18.021201,14.184755,7.065355,2.018673,...,0.0,1250.61,1250.61,0.0,0.0,0.000000e+00,1.0,0.0,1250.61,1.0
10,Obfelden,5721.0,22.087068,758.753316,23.579794,59.639923,16.780283,21.464779,4.946996,1.943463,...,0.0,1557.58,1557.58,2.0,0.0,9.793433e+03,3.0,0.0,1557.58,1.0
100,Stadel,2293.0,15.983814,177.889837,21.020497,61.971217,17.008286,14.086350,6.547359,1.745962,...,0.0,493.54,493.54,0.0,0.0,0.000000e+00,1.0,0.0,493.54,1.0
1001,Doppleschwand,789.0,9.888579,113.525180,26.742712,58.681876,14.575412,4.689480,5.092298,1.273074,...,0.0,595.00,595.00,0.0,0.0,0.000000e+00,0.0,0.0,595.00,1.0
1002,Entlebuch,3280.0,-0.545785,57.644991,21.798780,59.664634,18.536585,7.012195,4.875076,1.218769,...,0.0,8193.36,8193.36,0.0,0.0,0.000000e+00,2.0,0.0,8193.36,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,Walliswil bei Wangen,606.0,3.412969,197.394137,15.676568,63.036304,21.287129,4.290429,3.294893,0.000000,...,0.0,401.71,401.71,0.0,0.0,0.000000e+00,0.0,0.0,401.71,1.0
992,Wangen an der Aare,2321.0,13.330078,444.636015,19.000431,61.826799,19.172770,15.381301,5.180229,3.885172,...,0.0,2427.11,2427.11,0.0,0.0,1.081781e+04,3.0,0.0,2427.11,1.0
993,Wangenried,405.0,2.015113,139.175258,20.000000,56.296296,23.703704,4.444444,2.409639,0.000000,...,0.0,292.78,292.78,0.0,0.0,0.000000e+00,0.0,0.0,292.78,1.0
995,Wiedlisbach,2378.0,9.032554,317.066667,18.250631,63.456686,18.292683,17.872161,4.206099,1.261830,...,0.0,1469.56,1469.56,0.0,0.0,0.000000e+00,1.0,0.0,1469.56,1.0


In [31]:
# Save to data folder
final.to_csv(os.path.join(folder_data, "combined.csv"), index_label="BFS_NUMMER")
